<a href="https://colab.research.google.com/github/marta4cod/NNProject2/blob/main/NN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
!find /content/drive/MyDrive -name "NN2.ipynb"

In [4]:
# Set Git config
!git config --global user.name "marta4cod"
!git config --global user.email "marta4cod@gmail.com"

# Define GitHub info (DON'T COMMIT THIS TOKEN!)

username = "marta4cod"
repo = "NNProject2"

clone_url = f"https://{token}@github.com/{username}/{repo}.git"


In [16]:
# Clone the repo
!git clone {clone_url}

# Move the notebook (replace path below with the one you found earlier)
!mv "/content/drive/MyDrive/Colab Notebooks/NN2.ipynb" /content/{repo}/


fatal: destination path 'NNProject2' already exists and is not an empty directory.


In [22]:
%cd /content/NNProject2
!git add NN2.ipynb
!git commit --amend --no-edit  # replace the previous commit with this clean version
!git push --force origin main

/content/NNProject2
[main 762ded1] Add notebook from Colab
 Date: Thu Apr 3 12:01:04 2025 +0000
 1 file changed, 1 insertion(+)
 create mode 100644 NN2.ipynb
Enumerating objects: 3, done.
Counting objects: 100% (3/3), done.
Delta compression using up to 2 threads
Compressing objects: 100% (2/2), done.
Writing objects: 100% (3/3), 1.59 KiB | 1.59 MiB/s, done.
Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
To https://github.com/marta4cod/NNProject2.git
 * [new branch]      main -> main


In [ ]:
pip install datasets